# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [1]:
! pip install pandas
! pip install regex
! pip install matplotlib
! pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 88.2 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
  Using cached numpy-1.23.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.2/768.2 kB 16.8 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.34.4-py3-none-any.whl (944 kB)
  Using cached Pillow-9.2.0-cp38-cp38-manylinux_2_28_x86_64.whl (3.2 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[n

In [2]:
! pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.8.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.6 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [99]:

import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords



In [4]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [5]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [6]:
#Vamos a pasar a 0 y 1 la variable objetivo

df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [7]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [8]:
df_raw['url'].sample(20)

2109                            http://elv.fnd.to/crownes
2587                  https://deceuvel.nl/en/organisatie/
1226    https://www.vox.com/2020/6/22/21293168/dc-stat...
2797    https://www.reddit.com/r/aww/comments/hiz3bh/s...
1978    https://www.penguinrandomhouse.com/books/57678...
1195                   https://www.stophateforprofit.org/
2623    https://www.morningbrew.com/daily/stories/2020...
1419    https://www.theringer.com/2020/6/18/21294510/b...
2562    https://abcnews.go.com/US/george-floyds-relati...
2505    https://docs.google.com/forms/d/e/1FAIpQLSdkbW...
740                    https://briefingday.com/m/v4n3i4f3
277                     https://stevebryant.substack.com/
929     https://coronavirus.jhu.edu/testing/individual...
2734    https://www.washingtonpost.com/politics/2020/0...
317     https://en.wikipedia.org/wiki/Slab_City,_Calif...
382                           https://briefingday.com/fan
2202    https://www.morningbrew.com/daily/stories/2020...
2432          

In [14]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [15]:
# Vamos a ver como esta nuestra variable objetivo en este caso es is_spam
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("Que no son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)


Hay spam:  696
Que no son spam :  2303
(2999, 2)


In [16]:
# Eliminar los duplicados

df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

Empezamos el proceso de Limipieza de los datos

In [50]:
# colocar todos los textos en minusculas
df_raw['url'] = df_raw['url'].str.lower()

In [51]:
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

['i', 'me', 'my', 'myself', 'we']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
df_aux = df_raw.copy()

In [53]:

# Proceso de limpieza de datos

limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #savar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#assign the cleaned descriptions to the data frame
df_aux['url'] = limpieza
        
df_aux.head()

,url,is_spam
0,https briefingday us list manage com unsubscribe,1
1,https www hvper com,1
2,https briefingday com m v n i f,1
3,https briefingday com n m commentform,0
4,https briefingday com fan,1


In [58]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

https          2336
the             354
html            296
news            274
a               252
us              248
of              173
coronavirus     172
e               150
org             146
c               136
article         131
b               124
and             113
morningbrew     105
story           105
nytimes         101
on              101
daily            99
d                98
stories          94
utm              90
youtube          89
v                89
trump            88
numlock          87
watch            86
f                78
new              76
p                69
world            68
substack         68
reuters          65
covid            63
s                62
index            61
briefingday      61
en               59
vox              59
articles         58
iduskbn          58
cnn              58
co               56
politics         56
cnbc             54
n                54
sunday           51
business         49
court            48
apnews           47


In [74]:
# Creamos una lista de stopwords
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [75]:

def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result 


In [76]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [77]:
df_aux.sample(10)

,url,is_spam
998,fingeronthe app,0
24,josefschulz uebergang,0
111,cnn tech wirecard fraud tech accounting,0
1765,wsj articles shift plans go public year covid ...,0
1193,wtf us list manage unsubscribe,1
768,cnbc new york new jersey connecticut impose da...,0
1558,wsj articles fracking trailblazer chesapeake e...,0
1265,theguardian society jun online harassment incr...,0
2123,youtube watch n cp zpbmaq,0
1867,vox policy politics voxcare remdesivir price c...,0


In [78]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

html           296
news           274
us             248
coronavirus    172
org            146
article        131
story          105
morningbrew    105
nytimes        101
daily           99
stories         94
utm             90
youtube         89
trump           88
numlock         87
watch           86
new             76
substack        68
world           68
reuters         65
covid           63
index           61
briefingday     61
vox             59
en              59
articles        58
cnn             58
iduskbn         58
politics        56
co              56
cnbc            54
n               54
sunday          51
business        49
court           48
apnews          47
email           46
u               46
facebook        46
health          45
supreme         41
bbc             41
blog            40
medium          39
black           39
npr             38
police          38
digg            37
campaign        37
cases           35
theverge        35
apple           35
theguardian 

Esto es algo que estaba en el material de estudio para poder probar como funciona

df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)

In [79]:
# Ahora que hemos limpiado nuestros datos
# procedemos a hacer una copia con el dataset a trabajar en el final
df = df_aux.copy()

In [80]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [81]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [82]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [83]:
X_train.shape

(1776, 4819)

In [84]:
X_test.shape

(593, 4819)

In [85]:
print(vec.get_feature_names_out())

['aa' 'aab' 'aaron' ... 'zulalimtm' 'zwift' 'zwn']


In [88]:
nb = MultinomialNB()

In [90]:
nb.fit(X_train, y_train)

MultinomialNB()

In [91]:
nb.score(X_train,y_train)

0.956081081081081

In [92]:
nb.score(X_test,y_test)

0.9173693086003373

In [93]:
predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       532
           1       0.68      0.38      0.48        61

    accuracy                           0.92       593
   macro avg       0.80      0.68      0.72       593
weighted avg       0.91      0.92      0.91       593



In [94]:
#Creando la Matriz

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.40, random_state = 121, shuffle = True)

In [105]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [106]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       847
           1       0.71      0.51      0.60       101

    accuracy                           0.93       948
   macro avg       0.83      0.75      0.78       948
weighted avg       0.92      0.93      0.92       948



In [107]:
pickle.dump(cl, open('../models/texto_NLP.pkl', 'wb'))

In [108]:
# Si queremos cargar el archivo guardado en la carpeta models

load_model = pickle.load(open('../models/texto_NLP.pkl', 'rb'))

In [109]:
load_model

SVC(degree=4, gamma='auto', kernel='linear')